# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   506.636479  0.667760  1234  507.304238  505.968719  1014.608477   
6   152.301764  0.023577  1234  152.325340  152.278187   304.650681   
7   185.361621  0.696086  1234  186.057707  184.665535   372.115415   
8    49.845530 -0.249583  1234   49.595947   50.095113    99.191895   
9   690.964246  0.610607  1234  691.574853  690.353639  1383.149706   
10  167.512794  0.388685  1234  167.901479  167.124109   335.802958   
11  556.228101 -0.969469  1234  555.258632  557.197570  1110.517264   
12  642.568237  0.413190  1234  642.981427  642.155047  1285.962853   
13   18.979329  0.235460  1234   19.214789   18.743870    38.429579   
14  766.412698 -0.561206  1234  765.851491  766.973904  1531.702983   

              f  
5   1011.937439  
6    304.556374  
7    369.331070  
8    100.190226  
9   1380.707278  
10   334.248218  
11  1114.395141  
12  1284.310094  
13    37.487739  
14  1533.947809

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -507.304238   505.968719
6   -152.325340   152.278187
7   -186.057707   184.665535
8    -49.595947    50.095113
9   -691.574853   690.353639
10  -167.901479   167.124109
11  -555.258632   557.197570
12  -642.981427   642.155047
13   -19.214789    18.743870
14  -765.851491   766.973904

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -507.304238   505.968719
6   -152.325340   152.278187
7   -186.057707   184.665535
8    -49.595947    50.095113
9   -691.574853   690.353639
10  -167.901479   167.124109
11  -555.258632   557.197570
12  -642.981427   642.155047
13   -19.214789    18.743870
14  -765.851491   766.973904

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-507.30423839,  505.9687193 ],
       [-152.3253404 ,  152.27818685],
       [-186.05770743,  184.66553502],
       [ -49.59594744,   50.09511276],
       [-691.57485303,  690.35363901],
       [-167.901479  ,  167.12410911],
       [-555.25863218,  557.19757036],
       [-642.98142664,  642.15504712],
       [ -19.21478929,   18.74386963],
       [-765.85149136,  766.97390434]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1012.608477  -1011.937439
6     302.650681   -304.556374
7     370.115415   -369.331070
8      97.191895   -100.190226
9    1381.149706  -1380.707278
10    333.802958   -334.248218
11   1108.517264  -1114.395141
12   1283.962853  -1284.310094
13     36.429579    -37.487739
14   1529.702983  -1533.947809

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf